### when you want to define data type for certain columns then you can use schemaHints

In [0]:
landing_zone = '/Volumes/workspace/default/lateflow_connect_volume'
orders_data = landing_zone + '/orders_data'
checkpoint_path = landing_zone + '/manuallyInferschemacheckpoint'

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaHints", "order_id bigint, order_date string")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
orders_df.printSchema();

In [0]:
%sql
drop table if exists ordersdeltastream;
create table ordersdeltastream;

### when you ingest mismatch file then you need to refresh your data frame.

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;

### add new metad_data colmun

In [0]:
from pyspark.sql.functions import *



orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.schemaHints", "order_id bigint, order_date string")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data)
.withColumn("filename", col("_metadata.file_path"))
.withColumn("load_date", current_timestamp()))

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;
